# 1. 기본 프롬프트

In [16]:
from langchain.llms import OpenAI
import os
import config as cfg

os.environ['OPENAI_API_KEY'] = cfg.gpt_config["api_key"]

llm = OpenAI(model_name='text-davinci-003', temperature=0.9)
print(llm('1980년대 메탈 음악 5곡 추천해줘.'))



1. Guns N' Roses - Paradise City 
2. Metallica - Enter Sandman 
3. Bon Jovi - Livin' On A Prayer 
4. Judas Priest - Painkiller 
5. AC/DC - Back In Black


# Load chroma DB

In [2]:
import chromadb
client = chromadb.PersistentClient(path="chromadb")
    
collection1 = client.get_or_create_collection(
    name="law_table",
    metadata={"hnsw:space": "cosine"}
)

collection2 = client.get_or_create_collection(
    name="detail_law_table",
    metadata={"hnsw:space": "cosine"}
)

query_results_1 = collection1.query(
    query_texts=["인권"],
    n_results=5,
    where={"카테고리":"인권"},
)
    
query_results_2 = collection2.query(
    query_texts=["인권"],
    n_results=5,
    where={"카테고리":"인권"},
)

   
documents_list1 = [doc for sublist in query_results_1['documents'] for doc in sublist]
documents_list2 = [doc for sublist in query_results_2['documents'] for doc in sublist]
    
list1 = str(documents_list1)
list2 = str(documents_list2)

# GPT 3-5

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content="당신은 음악 추천을 해주는 전문 AI입니다.")
msg = HumanMessage(content='1980년대 메탈 음악 5곡 추천해줘.')

aimsg = chat([sys, msg])
print(aimsg.content)

물론입니다! 1980년대 메탈 음악의 대표적인 곡 5곡을 추천해 드리겠습니다:

1. Iron Maiden - "The Number of the Beast"
2. Judas Priest - "Breaking the Law"
3. Metallica - "Master of Puppets"
4. Black Sabbath - "Paranoid"
5. Slayer - "Angel of Death"

이 곡들은 1980년대 메탈 음악의 다양한 스타일과 풍미를 대표하는 곡들입니다. 즐겁게 감상해보시기 바랍니다!


# 2. 프롬프트 템플릿

In [3]:
from langchain.prompts import PromptTemplate

'''
prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로"
)

prompt.format(상품="AI 여행 추천서비스")
'''
prompt = PromptTemplate(
    input_variables=["상품"],
    template="{기관}의 최신 안건을 잘 정리해줘"
)

prompt.format(기관="국회")

'국회의 최신 안건을 정리해줘'

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat, prompt=prompt)

print(chain.run(기관=list1))

- 장애인권리보장법안(대안)(보건복지위원장)
  - 의안 일자: 2023-11-23
  - 의안번호: DD11479
  - 단계: 접수
  - 세부단계: 제안
  - 소관위원회: -

- 희귀질환관리법 일부개정법률안(대안)(보건복지위원장)
  - 의안 일자: 2023-11-23
  - 의안번호: DD11482
  - 단계: 접수
  - 세부단계: 제안
  - 소관위원회: -

- 주민등록법 일부개정법률안(대안)(행정안전위원장)
  - 의안 일자: 2023-11-23
  - 의안번호: DD11488
  - 단계: 접수
  - 세부단계: 제안
  - 소관위원회: -

- 병역법 일부개정법률안(대안)(국방위원장)
  - 의안 일자: 2023-11-23
  - 의안번호: DD11476
  - 단계: 접수
  - 세부단계: 제안
  - 소관위원회: -

- 장애인차별금지 및 권리구제 등에 관한 법률 일부개정법률안(노웅래의원 등 10인)
  - 의안 일자: 2023-11-22
  - 의안번호: 2125551
  - 단계: 위원회 심사
  - 세부단계: 회부
  - 소관위원회: 보건복지위원회


# ChatPromptTemplete & Chain
위 프롬프트 템플릿과 동일하지만 더 많은 기능을 사용할 수 있다.
system prompt, user prompty를 구분해서 사용할 수 있으며, input에 대한 언어와 output에 대한 언어를 설정할 수 있다.
run할 때는 input_language, output_language, text 3개의 값을 넘긴다. 

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import(
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

chat = ChatOpenAI(temperature=0)

template="입력으로 받은 국회의 최신 안건들을 잘 정리해줘"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chatchain = LLMChain(llm=chat, prompt=chat_prompt)
print(chatchain.run(text=list2))

국회의 최신 안건들은 다음과 같습니다:

1. 제대군인지원에 관한 법률 일부개정법률안
   - 제안일: 2023-06-20
   - 의결결과: 원안가결
   - 총투표수: 241.0, 찬성: 240.0, 반대: nan, 기권: 1.0
   - 위원회심사 의결일: 2023-04-06, 본회의심의 의결일: 2023-06-21, 정부이송일: 2023-06-30, 공포일: 2023-07-11
   - 링크URL: [링크 바로가기](https://likms.assembly.go.kr/bill/billDetail.do?billId=PRC_X2N3P0J4I0V3C1D3F3K1X4F8N4A8Y8)

2. 주민등록법 일부개정법률안
   - 제안일: 2023-07-26
   - 의결결과: 원안가결
   - 총투표수: 244.0, 찬성: 242.0, 반대: nan, 기권: 2.0
   - 위원회심사 의결일: 2023-03-22, 본회의심의 의결일: 2023-07-27, 정부이송일: 2023-08-04, 공포일: 2023-08-16
   - 링크URL: [링크 바로가기](https://likms.assembly.go.kr/bill/billDetail.do?billId=PRC_V2E3J0X3W2F1X1T4L5Y8Z2G1R6U0H8)

3. 여수ㆍ순천 10ㆍ19사건 진상규명 및 희생자 명예회복에 관한 특별법 일부개정법률안
   - 제안일: 2023-07-26
   - 의결결과: 원안가결
   - 총투표수: 240.0, 찬성: 215.0, 반대: 4.0, 기권: 21.0
   - 위원회심사 의결일: 2023-04-25, 본회의심의 의결일: 2023-07-27, 정부이송일: 2023-08-04, 공포일: 2023-08-16
   - 링크URL: [링크 바로가기](https://likms.assembly.go.kr/bill/billDetail.do?billId=PRC_U2C3R0W4F2M4U1M7B2X9J3Z7S5M7P5)

4. 장애인 건강권 및 의료접근성 보장에 관한 법률 일부개정법률안